In [3]:
import pandas as pd
import numpy as np

In [116]:
data = pd.read_csv("../Licencjat/licencjat.csv", delimiter=";")  

C:\Users\Ester\AppData\Local\Temp\ipykernel_11716\452193696.py:1: DtypeWarning: Columns (21,95,107) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../Licencjat/licencjat.csv", delimiter=";")


In [117]:
data = data[[
    "mergeid", "age", "isced1997_r", "mar_stat", "hhsize", "partnerinhh", "ch001_", "sphus",
    "chronic_mod", "eurod", "ep005_", "ep009_mod", "ep013_mod", "ep026_mod",
    "co007_", "thinc_m", "income_pct_w8", "wave"
]]

In [118]:
data = data[data['wave'] == 8]
data.head()

,mergeid,age,isced1997_r,mar_stat,hhsize,partnerinhh,ch001_,sphus,chronic_mod,eurod,ep005_,ep009_mod,ep013_mod,ep026_mod,co007_,thinc_m,income_pct_w8,wave
12,AT-001215-01,"80,8000030517578",5,4,1,3,0,4,5,2,1,-9,-9,-9,3,"17603,890625",3,8
17,AT-001492-01,68,3,1,2,1,6,3,2,1,5,-9,-9,-9,3,"26578,423828125",5,8
23,AT-001492-02,"68,3000030517578",4,1,2,1,6,2,3,2,1,-9,-9,-9,3,"26578,423828125",5,8
34,AT-001881-01,"89,6999969482422",1,6,2,3,3,5,3,-15,5,-9,-9,-9,-12,"16043,3388671875",2,8
40,AT-001881-02,"95,3000030517578",1,1,3,3,2,3,2,2,1,-9,-9,-9,3,"14446,015625",2,8


In [119]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53695 entries, 12 to 488396
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   mergeid        53695 non-null  object
 1   age            53695 non-null  object
 2   isced1997_r    53695 non-null  int64 
 3   mar_stat       53695 non-null  int64 
 4   hhsize         53695 non-null  int64 
 5   partnerinhh    53695 non-null  int64 
 6   ch001_         53695 non-null  int64 
 7   sphus          53695 non-null  int64 
 8   chronic_mod    53695 non-null  int64 
 9   eurod          53695 non-null  int64 
 10  ep005_         53695 non-null  int64 
 11  ep009_mod      53695 non-null  int64 
 12  ep013_mod      53695 non-null  object
 13  ep026_mod      53695 non-null  int64 
 14  co007_         53695 non-null  int64 
 15  thinc_m        53695 non-null  object
 16  income_pct_w8  53695 non-null  int64 
 17  wave           53695 non-null  int64 
dtypes: int64(14), object(4)
memor

In [120]:
data["ep013_mod"] = data["ep013_mod"].str.replace(",", ".")
data["thinc_m"] = data["thinc_m"].str.replace(",", ".")
data["age"] = data["age"].str.replace(",", ".")

In [128]:
data["age"] = data["age"].str.replace(",", ".").astype(float)
data["ep013_mod"] = data["ep013_mod"].str.replace(",", ".").astype(float)
data["thinc_m"] = data["thinc_m"].str.replace(",", ".").astype(float)

In [130]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53695 entries, 12 to 488396
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   mergeid        53695 non-null  object 
 1   age            53695 non-null  float64
 2   isced1997_r    53695 non-null  int64  
 3   mar_stat       53695 non-null  int64  
 4   hhsize         53695 non-null  int64  
 5   partnerinhh    53695 non-null  int64  
 6   ch001_         53695 non-null  int64  
 7   sphus          53695 non-null  int64  
 8   chronic_mod    53695 non-null  int64  
 9   eurod          53695 non-null  int64  
 10  ep005_         53695 non-null  int64  
 11  ep009_mod      53695 non-null  int64  
 12  ep013_mod      50577 non-null  float64
 13  ep026_mod      53695 non-null  int64  
 14  co007_         53695 non-null  int64  
 15  thinc_m        53695 non-null  float64
 16  income_pct_w8  53695 non-null  int64  
 17  wave           53695 non-null  int64  
dtypes: float6

In [90]:
data.shape

(53695, 18)

In [134]:
data = data[data['mergeid'].str.contains("PL", na=False)]

In [136]:
data.shape

(2856, 18)

In [138]:
data = data[(data >= 0).all(axis=1)]

TypeError: '>=' not supported between instances of 'str' and 'int'